In [1]:
!git clone https://github.com/Kotzly/DS4H_Course.git
!pip install ./DS4H_Course

fatal: destination path 'DS4H_Course' already exists and is not an empty directory.
Processing ./DS4H_Course
  Created wheel for ds4h: filename=ds4h-0.1-cp37-none-any.whl size=11900 sha256=46798d6750475a4d770f12bb4945cfff67152e35af41cb91ed00d124f76e0579
  Stored in directory: /tmp/pip-ephem-wheel-cache-78qojp08/wheels/ec/ff/91/fc3664f800ce643b3917437b9c8d3b0b7298db38297e3d6ef4
Successfully built ds4h
  Found existing installation: ds4h 0.1
    Uninstalling ds4h-0.1:
      Successfully uninstalled ds4h-0.1


In [2]:
import ds4h
from ds4h.data.download_sinasc import download_dbc, download_dbf_2020
from ds4h.processing.dbc import dbc_to_csv, dbf_to_csv
from ds4h.processing.csv import join_sinasc_files
from ds4h.processing.sinasc import process_sinasc



from pathlib import Path
from ds4h.data.city_ranking import get_ranking_dataframe, get_top_cities_from_df

In [3]:
dataset_root = Path("/home/paulo/Documents/datasets/sinasc")

In [4]:
dbc_path = dataset_root / "dbc"

dbc_path.mkdir(exist_ok=True, parents=True)

In [ ]:
download_dbc(dbc_path)

In [ ]:
csv_path = dataset_root / "csv"
dbc_to_csv(dbc_path, csv_path);

In [ ]:
dbf_folder = dataset_root / "dbf"
dbf_folder.mkdir(exist_ok=True)

download_dbf_2020(dbf_folder)

dbf_filepath = list(dbf_folder.glob("*.DBF"))[0]
dbf_to_csv(dbf_filepath, csv_path)

In [ ]:
union_df = join_sinasc_files(csv_path)
union_df

In [ ]:
df = get_ranking_dataframe()
cities_dict = get_top_cities_from_df(df)
print(cities_dict)

In [ ]:
df = process_sinasc(union_df)

In [ ]:
import numpy as np
from scipy import stats
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import shutil
import urllib.request as request
from contextlib import closing
import zipfile
import sys
import os
from simpledbf import Dbf5
import platform



#Verifica dados faltantes

In [ ]:
union_df.shape

In [ ]:
faltantes = union_df.isnull().sum()
print(faltantes)

In [ ]:
def missing_values(x):
  missing_per = pd.isnull(x).mean()
  return "{:.2f}%".format(missing_per * 100)

union_df.apply(missing_values, axis=0)

### Considerando que os dados faltantes do dataset é menor que 5% optamos por dropar os dados

In [ ]:
union_df = union_df.dropna()





## Qualidade dos dados 
###Verifica erros de registro, valores discrepantes e Inconsistências

In [ ]:
union_df.dtypes

In [ ]:
union_df.describe().transpose()

#Tratamento da Variavel DTNASC.

In [ ]:
from datetime import datetime as dt
def str_to_datetime(date):
  string = str(date)
  # 3022017 gives error (it reads the day as 30, but the month 22 does not exist), so we add a 0 to the start of the string
  if (string[0] != "0") and len(string) < 8:
    string = "0" + string
  return dt.strptime(string, "%d%m%Y")

union_df.loc[:, "DTNASC"] = union_df["DTNASC"].apply(str_to_datetime).astype("datetime64")

Tratamento da variável QTDFILVIVO, QTDFILMORT, IDADEMAE.
Essas variáveis precisam ser tratadas para remover os valores discrepantes.

In [ ]:
union_df.groupby(by=["QTDFILVIVO"]).count()['DTNASC']

In [ ]:
union_df = union_df[union_df['QTDFILVIVO']<=30]


In [ ]:
union_df.groupby(by=["QTDFILMORT"]).count()['DTNASC']

In [ ]:
union_df = union_df[union_df['QTDFILMORT']<=30]

In [ ]:
union_df.groupby(by=["IDADEMAE"]).count()['DTNASC']

In [ ]:
union_df = union_df[union_df['QTDFILMORT']<=65]

In [ ]:
union_df.groupby(by=["RACACOR"]).count()['DTNASC']

De acordo com o dicionário de dados essa variável deveria ser Tipo de raça e cor do nascido: 1– Branca; 2– Preta; 3– Amarela;4– Parda; 5– Indígena. Então esse 9 pode ser valores que não foram definidos. Vamos remover do dataset.

In [ ]:
union_df = union_df[union_df['RACACOR']<=5]

In [ ]:
union_df.groupby(by=["RACACOR"]).count()['DTNASC']

In [ ]:
union_df.groupby(by=["ESTCIVMAE"]).count()['DTNASC']

De acordo com o dicionário de dados essa variável deveria: 1– Solteira; 2– Casada; 3– Viúva; 4– Separada judicialmente/divorciada; 5– União estável; 9– Ignorada. Vamos filtrar os valores ignorados tb

In [ ]:
union_df = union_df[union_df['ESTCIVMAE']<=5]

In [ ]:
union_df.groupby(by=["ESTCIVMAE"]).count()['DTNASC']

In [ ]:
union_df.groupby(by=["PARTO"]).count()['DTNASC']

Apenas 66 mulheres tiveram o tipo de parto ignorado na hora de transcrever os dados. Vamos filtrar

In [ ]:
union_df = union_df[union_df['PARTO'] <= 2.0]

In [ ]:
union_df.groupby(by=["PARTO"]).count()['DTNASC']

In [ ]:
union_df.groupby(by=["IDANOMAL"]).count()['DTNASC']

In [ ]:
union_df = union_df[union_df['IDANOMAL'] <= 2.0]

In [ ]:
union_df.groupby(by=["IDANOMAL"]).count()['DTNASC']

In [ ]:
union_df.groupby(by=["GESTACAO"]).count()['DTNASC']

Semanas de gestação: 1– Menos de 22 semanas; 2– 22 a 27 semanas; 3– 28 a 31 semanas; 4– 32 a 36 semanas; 5– 37 a 41 semanas; 6– 42 semanas e mais; 9– Ignorado.

In [ ]:
union_df = union_df[union_df['GESTACAO'] <= 6.0]

In [ ]:
union_df.groupby(by=["GESTACAO"]).count()['DTNASC']

In [ ]:
union_df.groupby(by=["ESCMAE2010"]).count()['DTNASC']

In [ ]:
union_df = union_df[union_df['ESCMAE2010'] <= 5.0]

In [ ]:
union_df.groupby(by=["ESCMAE2010"]).count()['DTNASC']